# Build a Chatbot

* Conversational RAG
* Agents

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BABB36EDE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BABB36F9B0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Abhi and I am AI developer")])

AIMessage(content="Hi Abhi, it's nice to meet you! \n\nThat's awesome that you're an AI developer. What kind of AI projects are you working on? I'm always interested to hear about what people are creating in the field.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 20, 'total_tokens': 74, 'completion_time': 0.098181818, 'prompt_time': 7.7569e-05, 'queue_time': 0.020020651, 'total_time': 0.098259387}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7af5f601-e0e0-41ab-a90d-eeaa7872ce00-0', usage_metadata={'input_tokens': 20, 'output_tokens': 54, 'total_tokens': 74})

In [5]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Abhi and I am AI developer"),
    AIMessage(content="Hello Krish! It's nice to meet you .\nn\n As an AI developer, what kind of work you do?"),
    HumanMessage(content="Hey What's my name and what do I do?")
])

AIMessage(content="You said your name is Abhi and that you are an AI developer! 😊  \n\nIs there anything else you'd like to tell me about your work?  I'm curious to learn more.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 68, 'total_tokens': 112, 'completion_time': 0.08, 'prompt_time': 0.001824214, 'queue_time': 0.021721956, 'total_time': 0.081824214}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d477c9d8-93a2-47ab-81a7-5e908ae3b610-0', usage_metadata={'input_tokens': 68, 'output_tokens': 44, 'total_tokens': 112})

### Mesage History
To wrap our model and make it stateful. It keeps track of inputs and outputs of model and store them in some datastore

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
response = with_message_history.invoke([HumanMessage(content="Hi , My name is Abhijeet and I am an AI developer")],
                            config=config)

In [13]:
response.content

"Hi Abhijeet, \n\nIt's great to meet you!  Being an AI developer is really exciting.  What are you working on? \n\nI'm always interested to hear about the cool projects people are creating.  Are you working with any specific AI models or technologies?\n"

In [14]:
with_message_history.invoke([HumanMessage(content="What is my name?")],
                            config=config)

AIMessage(content='Your name is Abhijeet.  😊  I remember that from our first introduction! \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 264, 'total_tokens': 286, 'completion_time': 0.04, 'prompt_time': 0.00830962, 'queue_time': 0.02057022, 'total_time': 0.04830962}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1f683af6-d196-4c1d-b4fb-d04858417eb0-0', usage_metadata={'input_tokens': 264, 'output_tokens': 22, 'total_tokens': 286})

In [19]:
## change the config --> session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke([HumanMessage(content="What is my name?")],
                            config=config1)

response.content

'As an AI, I have no access to your personal information, including your name. To tell me your name, simply type it out! 😊\n'

In [20]:
response = with_message_history.invoke([HumanMessage(content="Hey my name is John")],
                            config=config1)

response.content

"Hi John, it's nice to meet you! 👋 \n\nIs there anything I can help you with today?\n"

In [21]:
response = with_message_history.invoke([HumanMessage(content="What is my name?")],
                            config=config1)

response.content

'You told me your name is John!  😊  \n\nDid you forget?  \n'